**2. mājasdarbs**

In [ ]:
# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021

# Parametri
LOAD_MODEL = True # karodziņš, kura True vērtība nozīmē, ka modeli ir jāieladē no faila; False - ka modeli ir jātrenē uz trenēšanas datiem
STATE_DICT_FILE = 'md2_model_jj19021.pt' # saglabātā modeļa faila nosaukums
TEST_FILE = 'dice_09.dat' # testa faila nosaukums no trenēšanas datu kopas

In [ ]:
# Trenēšanas un testēšanas datu ielāde
# Koda fragments paņemts no Google Colab piemēra "Dice loading PyTorch"
!wget 'https://www.dropbox.com/s/k2ynyl95bzzurti/dice.zip'
!unzip dice.zip
!rm -rf __MACOSX

In [ ]:
# Saglabātā modeļa ieguve
# Ar wget tiek ielādēts fails no Dropbox
!wget 'https://www.dropbox.com/s/buyze4tey0dnfm3/md2_model_jj19021.pt'

# Funkcija modeļa ielādēšanai, kas saņem state_dict no faila un ielādē to modelī
def load_model(model):
  state_dict = torch.load(STATE_DICT_FILE)
  model.load_state_dict(state_dict)

# Funkcija modeļa saglabāšanai, kas iegūst modeļa tekošo state_dict un ielādē to failā
def save_model(model):
  state_dict = model.state_dict()
  torch.save(state_dict, STATE_DICT_FILE)

In [ ]:
import io
import os
import torch
from torch import nn, optim
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Dataset realizācija
# Koda fragments paņemts no Google Colab piemēra "Dice loading PyTorch"
class DiceDataset(Dataset):
    def __init__(self, train=True, root_path='dice/data/', test_file=TEST_FILE):
      contents = []
      for f in os.scandir(root_path):
        if f.is_file() and f.name.endswith('.dat'):
          if train ^ (f.name == test_file):
            file_contents = np.fromfile(root_path+f.name, dtype=np.uint8).reshape(-1,401)
            # Katra bilde ir 401 baits, pirmajā baitā atbilde un tad 20x20 px
            contents.append(file_contents)
      contents = np.vstack(contents)
      self.pictures = contents[:,1:].reshape(-1,1,20,20) # Channel x Height x Width
      self.pictures = torch.tensor(self.pictures, dtype=torch.float32) / 255
      self.labels = contents[:,0]

    def __getitem__(self, key):
        return self.pictures[key], self.labels[key]

    def __len__(self):
        return len(self.pictures)

# Trenēšanas dati - karodzīņš train ir True, tiks izmantoti visi trenešanai paredzētie dati
train_data = DiceDataset(train=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
# Testēšanas dati - karodziņš train ir False, tiks izmantoti tikai testēšanai paredzetie dati - fails dice_09.dat
test_data = DiceDataset(train=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
# Datu attēli
# Koda fragments paņemts no Google Colab piemēra "Dice loading PyTorch"

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'gray'
def show_pictures(data):
  figure = plt.figure(figsize=(10, 5))
  for n in range(40):
    figure.add_subplot(4,10,n+1)
    img, label = data[n]
    plt.title(label)
    plt.imshow(img.squeeze())
    plt.axis('off')

show_pictures(test_data)

In [ ]:
# Tīkla modulis
# Tiek izmantots nn.Sequential realizācijas vienkāršošanai, jo visi slāņi tiks pielietoti secīgi
# Slāņi un nepieciešamie parametri tika noskaidroti, vairākkārt testējot modeli un skatoties uz iegūto precizitāti,
# kā piemēru sākotnēji lietojot MNIST modeļa realizāciju no Google Colab piemēra "PyTorch MNIST demo"
class DiceModel(nn.Sequential):
    def __init__(self):
      super().__init__(
          nn.Conv2d(1, 64, (5,5)),
          nn.ReLU(),
          nn.MaxPool2d((3,3)),
          nn.Flatten(),
          nn.Linear(in_features=1600, out_features=128),
          nn.ReLU(),
          nn.Linear(in_features=128, out_features=13),
          nn.LogSoftmax(dim=1)
        )

model = DiceModel()

In [ ]:
# Trenēšanas parametru konfigurācija
epochs = 5
lr = 0.01
momentum = 0.9

# GPU pieejamības pārbaude
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Ja modeļa ielādes karodziņš ir patiess, ielādē modeļa state_dict no faila
if LOAD_MODEL:
  load_model(model)
# Citādi notiek modeļa trenēšana
# Koda fragments paņemts no Google Colab piemēra "PyTorch MNIST demo"
else:
  for epoch in range(epochs):
      running_loss = 0.0

      for times, data in enumerate(train_loader):
          inputs, labels = data[0].to(device), data[1].to(device)

          # Notīram gradientus
          optimizer.zero_grad()

          # Foward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # Izdrukājam statistiku
          running_loss += loss.item()
          if times % 100 == 99 or times+1 == len(train_loader):
              print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(train_loader), running_loss/times))
                # running_loss / times - vidējā zudumfunkcija tekošā epohā

  print('Trenēšana ir pabeigta.')

In [ ]:
# Modeļa saglabāšana
save_model(model)

In [ ]:
# Testēšana un rezultāti
# Tiek izmantota testa datu kopa
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)  # Outputs ir vektors no 13 'varbūtībām'
        _, predicted = torch.max(outputs.data, 1)
          # Max rezultāta pirmā daļa ir maksimums, otrā - kurā vietā tas ir,
          # t.i. argmax, kura summa ar lielāko varbūtību
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          # (predicted == labels) tā kā abi ieejā ir vienāda garuma vektori, tad rezultāts ir tāda pat garuma vektors no nullītēm un vieniniekiem

print('Precizitāte uz testa datu kopas: %.2f %%' % (100 * correct / total))

# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021

Precizitāte uz testa datu kopas: 99.74 %
